## AI-HUB 데이터 우선 정제, 중립감정 활용할 데이터 있음. & 2차분류 데이터 미리 만들기

In [1]:
import pandas as pd
import plotly.express as px

In [146]:
df1 = pd.read_csv("./감성대화/감성대화말뭉치(최종데이터)_Training.csv", encoding='UTF-8')
df2 = pd.read_csv("./감성대화/감성대화말뭉치(최종데이터)_Validation.csv", encoding='UTF-8')

In [19]:
df1 = df1[['감정_대분류', '사람문장1']] # 감성대화중 감정 대분류에 해당하는 것은 '사람문장1'이기 때문에 '감정대분류'와 '사람문장1'만 남겨놓음.
df1.rename(columns={"감정_대분류": "sentiment", "사람문장1":"sentence"}, inplace=True) # column명 통일
df1 = df1[["sentence", "sentiment"]] # column순서 변경

# df2에 대해서도 동일한 과정 실시
df2 = df2[['감정_대분류', '사람문장1']]
df2.rename(columns={"감정_대분류": "sentiment", "사람문장1":"sentence"}, inplace=True)
df2 = df2[["sentence", "sentiment"]]

In [20]:
result = pd.concat([df1,df2])
result.drop_duplicates(inplace=True)
result.reset_index(drop=True, inplace=True)
result['sentiment'] = result['sentiment'].apply(lambda x: "분노" if x == "상처" else x) # 상처와 분노를 동종의 감정으로 간주.
result['sentiment'] = result['sentiment'].apply(lambda x: "불안" if x == "당황" else x) # 당황과 불안을 동종의 감정으로 간주.
result.to_csv("./AIhub_data1.csv", index=False, encoding='UTF-8')
result

,sentence,sentiment
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,분노
...,...,...
58251,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,불안
58252,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,불안
58253,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,불안
58254,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,불안


In [22]:
result.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [23]:
result['sentiment'].unique()

array(['분노', '기쁨', '불안', '슬픔'], dtype=object)

In [24]:
# "sentiment" 컬럼의 값들의 분포를 막대그래프로 그리기
fig = px.histogram(result, x='sentiment', title='Sentiment Distribution')

# fig의 크기 조절 (width와 height를 원하는 크기로 설정합니다.)
fig.update_layout(
    width=800,  # 가로 크기
    height=500,  # 세로 크기
    xaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # x축 레이블 폰트 크기와 스타일 조절
    yaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # y축 레이블 폰트 크기와 스타일 조절
)

# 색상을 랜덤으로 지정
# 'color_discrete_sequence' 파라미터에 'px.colors.qualitative'를 사용하면 Plotly에서 제공하는 랜덤 색상을 사용할 수 있습니다.
fig.update_traces(marker=dict(color=px.colors.qualitative.Prism))

# 막대그래프의 모양 변경 (width 속성을 여기서 조절)
fig.update_traces(
    marker=dict(color=px.colors.qualitative.Prism),  # 색상을 랜덤하게 지정
    marker_line_width=3,  # 테두리 선 두께 조절
    marker_line_color='black',  # 테두리 선 색상 조절
    opacity=0.9,  # 막대의 투명도 조절 (0부터 1 사이의 값을 사용, 1은 완전 불투명)
)

# 그래프 출력
sentiment_distribution = result['sentiment'].value_counts()

print(sentiment_distribution)
fig.show()

분노    20562
불안    20231
슬픔    10125
기쁨     7338
Name: sentiment, dtype: int64


In [131]:
df3 = pd.read_csv("./감정 분류를 위한 대화 음성 데이터셋/4차년도.csv", encoding='cp949')
df3.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

df4 = pd.read_csv("./감정 분류를 위한 대화 음성 데이터셋/5차년도.csv", encoding='cp949')
df4.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

df5 = pd.read_csv("./감정 분류를 위한 대화 음성 데이터셋/5차년도_2차.csv", encoding='cp949')
df5.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

In [132]:
def emo_sum(dataframe): # 각 row의 감정세기를 계산하는 함수
    rep_emo = []
    for index, row in dataframe.iterrows():
        emo = {}
        for i in dataframe.columns[3:13:2]:
            emo[row[i]] = emo.get(row[i], 0) + row[i+"세기"]
        # 딕셔너리의 값들 중에서 최댓값 찾기
        max_value = max(emo.values())

        # 최댓값에 해당하는 key 찾기
        max_key_list = [key for key, value in emo.items() if value == max_value]
        rep_emo.append(max_key_list[0])
    return rep_emo

In [133]:
df3["sentiment"] = emo_sum(df3) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df3 = df3[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df3.dropna(inplace=True)
df3.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일

df4["sentiment"] = emo_sum(df4) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df4 = df4[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df4.dropna(inplace=True)
df4.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일

df5["sentiment"] = emo_sum(df5) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df5 = df5[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df5.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일
df5.dropna(inplace=True)
df5['sentiment'] = df5['sentiment'].apply(lambda x : x.capitalize())

In [141]:
result2 = pd.concat([df3,df4,df5])
result2.drop_duplicates(inplace=True, keep='last')
result2.reset_index(drop=True, inplace=True)
result2['sentiment'] = result2['sentiment'].apply(lambda x: "중립" if x == "Neutral" else x)
result2['sentiment'] = result2['sentiment'].apply(lambda x: "분노" if x == "Fear" or x == "Disgust" else x) # Fear 와 Digust를 분노의 감정으로 간주
result2['sentiment'] = result2['sentiment'].apply(lambda x: "슬픔" if x == "Sadness" else x) # 한글로 통일
result2['sentiment'] = result2['sentiment'].apply(lambda x: "기쁨" if x == "Happiness" else x) # 한글로 통일
result2['sentiment'] = result2['sentiment'].apply(lambda x: "분노" if x == "Angry" else x) # 한글로 통일
result2 = result2[result2['sentiment']!= "Surprise"] # Surprise를 제거
result2.to_csv("./AIhub_data2.csv", index=False, encoding='UTF-8')
result2['sentiment'].value_counts()

슬픔    17059
분노    13879
기쁨     4736
중립      534
Name: sentiment, dtype: int64

In [142]:
result2.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [143]:
result2['sentiment'].unique()

array(['분노', '슬픔', '기쁨', '중립'], dtype=object)

In [144]:
# "sentiment" 컬럼의 값들의 분포를 막대그래프로 그리기
fig = px.histogram(result2, x='sentiment', title='Sentiment Distribution')

# fig의 크기 조절 (width와 height를 원하는 크기로 설정합니다.)
fig.update_layout(
    width=800,  # 가로 크기
    height=500,  # 세로 크기
    xaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # x축 레이블 폰트 크기와 스타일 조절
    yaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # y축 레이블 폰트 크기와 스타일 조절
)

# 색상을 랜덤으로 지정
# 'color_discrete_sequence' 파라미터에 'px.colors.qualitative'를 사용하면 Plotly에서 제공하는 랜덤 색상을 사용할 수 있습니다.
fig.update_traces(marker=dict(color=px.colors.qualitative.Prism))

# 막대그래프의 모양 변경 (width 속성을 여기서 조절)
fig.update_traces(
    marker=dict(color=px.colors.qualitative.Prism),  # 색상을 랜덤하게 지정
    marker_line_width=3,  # 테두리 선 두께 조절
    marker_line_color='black',  # 테두리 선 색상 조절
    opacity=0.9,  # 막대의 투명도 조절 (0부터 1 사이의 값을 사용, 1은 완전 불투명)
)

# 그래프 출력
fig.show()

In [57]:
sentiment_distribution = result2['sentiment'].value_counts()
print(sentiment_distribution)

슬픔    16309
분노    11932
기쁨     4016
열정     1173
중립      534
Name: sentiment, dtype: int64


In [148]:
import pandas as pd
data6 = pd.read_csv("사랑이별_커뮤니티_챗봇데이터_전처리완.csv", encoding='UTF-8')
data6['sentiment'].value_counts()

불안    3991
사랑    2631
중립    2004
슬픔    1632
기쁨    1565
Name: sentiment, dtype: int64

In [59]:
data7 = pd.read_csv("감정별_가사_정리_9class.csv", encoding='UTF-8')
data7

,Unnamed: 0,감정,가수,제목,가사
0,1,사랑,양요섭&정은지,LOVE DAY,나도 알아 날 괴롭히는 것도\n고민은 항상 날 무너트리네\n세상에 맘껏 날 쏟아부어...
1,2,사랑,SG워너비,해바라기,너는 나의 사랑 너는 나의 행복 소중한 사람\n비가 내리면 함께 맞아줄 둘도 없는 ...
2,3,사랑,SG워너비,가슴 뛰도록,사랑해 사랑해 사랑해 \n\n매일 아침 눈을 뜰 때마다 \n나를 설레게 하는 사람...
3,4,사랑,케이시,너의 발걸음에 빛을 비춰줄게,언제나 너의 곁에 있어 줄게\n눈부시게 빛나 모든 날들이\n세상 모두가 변한다 해도...
4,5,사랑,싹쓰리,여름 안에서,언제나 꿈꿔 온 순간이 여기 지금 내게 시작되고 있어\n그렇게 너를 사랑했던 내마음...
...,...,...,...,...,...
179,16,분노,K.hertz,습관,넌 내게 습관이 되어 머문다\n담배 같아 금단이 찾아 오잖아\n내 곁에 없는 그 매...
180,17,분노,jinsil,자격지심,솔직히 나 쪽팔려 \n이렇게 돼 버린 거 \n솔직히 나 열받아 \n뭐 하나 되는 게...
181,18,분노,유준성,Hate You,어짜피 니가 다시는 \n나를 사랑하지 않을꺼라면 \n그만 갖고 놀아 니 맘대로 \n...
182,19,분노,Old Clothez,집어치워,집어치워 꺼져버려 꺼져버려 집어치워\n집어치워 꺼져버려 꺼져버려 집어치워\n집어치워...


In [60]:
from tqdm import tqdm 
# 새로운 데이터프레임 생성
data7_1 = pd.DataFrame(columns=['sentence', 'sentiment'])

# 가사와 감정 정보를 "\n"으로 분리하여 새로운 데이터프레임에 추가
for index, row in tqdm(data7.iterrows()):
    lyrics = row['가사'].split('\n')
    emotion = row['감정']
    for sentence in lyrics:
        data7_1 = data7_1.append({'sentence': sentence, 'sentiment': emotion}, ignore_index=True)

184it [00:14, 12.32it/s]


In [61]:
data7_1

,sentence,sentiment
0,나도 알아 날 괴롭히는 것도,사랑
1,고민은 항상 날 무너트리네,사랑
2,세상에 맘껏 날 쏟아부어도,사랑
3,남아있는 건 나 하나,사랑
4,,사랑
...,...,...
10731,입속에서 맴돈 말,분노
10732,오늘은 할게,분노
10733,네가 죽었으면 좋겠어,분노
10734,네가 미쳤으면 좋겠어,분노


In [62]:
data7_1['sentiment'].value_counts()

열정      1912
분노      1569
신나는     1462
두려움     1175
외로움      998
사랑       952
행복       903
그리움      886
슬픔       814
신나는       65
Name: sentiment, dtype: int64

In [63]:
data7_1.drop_duplicates(inplace=True, keep='last')
data7_1.reset_index(drop=True, inplace=True)
data7_1['sentiment'] = data7_1['sentiment'].apply(lambda x: "열정" if x == "신나는" or x == "신나는 " else x)
data7_1['sentiment'] = data7_1['sentiment'].apply(lambda x: "불안" if x == "두려움" else x)
data7_1['sentiment'] = data7_1['sentiment'].apply(lambda x: "슬픔" if x == "외로움" else x) 
data7_1['sentiment'] = data7_1['sentiment'].apply(lambda x: "기쁨" if x == "행복" else x) 
data7_1['sentiment'] = data7_1['sentiment'].apply(lambda x: "슬픔" if x == "그리움" else x)

In [64]:
data7_1['sentiment'].value_counts()

열정    2052
슬픔    1650
분노     938
불안     729
사랑     616
기쁨     508
Name: sentiment, dtype: int64

In [149]:
data8 = pd.read_csv("korean_single_turn_utterance_180124.csv", encoding='UTF-8')
data8

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [150]:
data8 = data8[['Sentence', 'Emotion']] # 문장과 감정만 남겨놓음
data8 = data8[data8['Emotion'] != "놀람"]
data8.dropna(inplace=True)
data8.rename(columns={'Sentence':'sentence', 'Emotion': 'sentiment'}, inplace=True) # column명 통일
data8['sentiment'] = data8['sentiment'].apply(lambda x: "분노" if x == "혐오" or x == "공포" else x)
data8['sentiment'] = data8['sentiment'].apply(lambda x: "기쁨" if x == "행복" else x)

In [151]:
data8['sentiment'].value_counts()

분노    16562
기쁨     6037
슬픔     5267
중립     4830
Name: sentiment, dtype: int64

In [152]:
sec_clas = pd.concat([result,data6,data8])
print(sec_clas['sentiment'].value_counts())
sec_clas

분노    37124
불안    24222
슬픔    17024
기쁨    14940
중립     6834
사랑     2631
Name: sentiment, dtype: int64


,sentence,sentiment
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,분노
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,분노
38590,재미가 없으니 망하지,분노
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,분노
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,분노


In [153]:
sec_clas.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [154]:
# Mapping dictionary to convert emotions to positive or negative
emotion_map = {
    '사랑': '긍정',
    '기쁨': '긍정',
    '불안': '부정',
    '슬픔': '부정',
    '분노': '부정'
}

# Function to map emotions to positive or negative using the emotion_map
def map_sentiment(emotion):
    return emotion_map.get(emotion, emotion)

# Apply the map_sentiment function to the 'sentiment' column
sec_clas['sentiment'] = sec_clas['sentiment'].apply(map_sentiment)

In [155]:
sec_clas['sentiment'].value_counts()

부정    78370
긍정    17571
중립     6834
Name: sentiment, dtype: int64

In [156]:
sec_clas

,sentence,sentiment
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,부정
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,부정
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,부정
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,부정
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,부정
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,부정
38590,재미가 없으니 망하지,부정
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,부정
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,부정


## 긍정/부정/중립 1차 감정분류를 위한 학습데이터 전처리과정.

In [99]:
# 긍부정 라벨링된 데이터셋 정규화 과정
data1 = pd.read_csv("./multi_sentiment_dataset.csv", encoding='UTF-8') # AI hub에 있는 다감정 대화데이터셋인데 https://sig413.tistory.com/5 여기서 그냥 긍부정으로 나눠버림...
data1.rename(columns={'발화': 'sentence', "감정_int":"sentiment"}, inplace=True) # column명 통일

data2 = pd.read_csv("./ratings.txt", sep="\t", encoding='UTF-8') # 긍부정으로 라벨링된 네이버 영화리뷰 데이터셋 
data2 = data2[['document', 'label']] # 문장과 감정만 남겨놓음
data2.rename(columns={'document': 'sentence', "label":"sentiment"}, inplace=True) # column명 통일

data3 = pd.read_csv("./naver_shopping.txt", sep="\t", encoding='UTF-8') # 긍부정으로 라벨링된 네이버 쇼핑리뷰 데이터셋 
data3 = data3[['sentence', 'sentiment']] # columns 순서 변경
data3['sentiment'] = data3['sentiment'].apply(lambda x: 1 if x >= 4 else 0) # 'sentiment' column 값 변경 4~5점을 1(긍정)으로 1~2점을 0(부정)으로 변환

data4 = pd.read_csv("./steam.txt", sep="\t", encoding='UTF-8') # 긍부정으로 라벨링된 스팀 한국어 리뷰 데이터셋
data4 = data4[['sentence', 'sentiment']] # columns 순서 변경

# data5 = pd.read_csv("./chatbot_dataset.csv", encoding='UTF-8') # 챗봇 트레이닝용 문답 데이터셋, 답어가 중립감정이라 생각하여 활용할 예정. 나머지는 다감정분석에 활용할 것.
# data5 = data5[['A', 'label']] # 활용할 columns만 남겨놓음
# data5.rename(columns={'A': 'sentence', "label":"sentiment"}, inplace=True) # column명 통일
# data5['sentiment'] = data5['sentiment'].apply(lambda x: 2) # 중립인 감정의 'sentiment' 값을 2로 함

data6 = pd.read_csv("./AIhub_data2.csv", encoding='UTF-8') # AI hub에 있는 다감정 대화데이터셋
data6 = data6[data6["sentiment"] == "Neutral"]
data6['sentiment'] = data6['sentiment'].apply(lambda x: 2) # 중립인 감정의 'sentiment' 값을 2로 함

result3 = pd.concat([data1,data2,data3,data4,data6])

In [100]:
result3['sentiment'] = result3['sentiment'].apply(lambda x: "부정" if x == 0 else "중립" if x == 2 else "긍정")

In [101]:
result3

,sentence,sentiment
0,언니 동생으로 부르는게 맞는 일인가요..??,부정
1,그냥 내 느낌일뿐겠지?,부정
2,아직너무초기라서 그런거죠?,부정
3,유치원버스 사고 낫다던데,부정
4,근데 원래이런거맞나요,부정
...,...,...
99995,한글화해주면 10개산다,부정
99996,개쌉노잼 ㅋㅋ,부정
99997,노잼이네요... 30분하고 지웠어요...,부정
99998,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.,긍정


In [157]:
result3['sentiment'].value_counts()

부정    257100
긍정    257025
Name: sentiment, dtype: int64

In [158]:
result3.isnull().sum()

sentence     8
sentiment    0
dtype: int64

In [159]:
result4 = pd.concat([result3,sec_clas])
result4

,sentence,sentiment
0,언니 동생으로 부르는게 맞는 일인가요..??,부정
1,그냥 내 느낌일뿐겠지?,부정
2,아직너무초기라서 그런거죠?,부정
3,유치원버스 사고 낫다던데,부정
4,근데 원래이런거맞나요,부정
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,부정
38590,재미가 없으니 망하지,부정
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,부정
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,부정


In [160]:
result4['sentiment'].value_counts()

부정    335470
긍정    274596
중립      6834
Name: sentiment, dtype: int64

In [161]:
result4.dropna(axis=0, inplace=True)
result4.reset_index(drop=True, inplace=True)
result4.to_csv("./Ternary_classification_data.csv", index=False, encoding='UTF-8')

## 긍정/부정/중립 감정단어사전 제작 (군산대학교 랩실 데이터)

In [55]:
import pandas as pd

In [56]:
dic1 = pd.read_csv("neg_pol_word.txt", sep='\t', encoding='UTF-8')
dic1['sentiment'] = dic1['sentiment'].apply(lambda x : "부정")

dic2 = pd.read_csv("pos_pol_word.txt", sep='\t', encoding='UTF-8')
dic2['sentiment'] = dic2['sentiment'].apply(lambda x : "긍정")

dic3 = pd.read_csv("SentiWord_Dict.txt", sep='\t', encoding='UTF-8')
dic3 = dic3[dic3['sentiment'] != 0.0]
dic3['sentiment'] = dic3['sentiment'].apply(lambda x : "긍정" if x > 0 else "부정" if x < 0 else x)

In [57]:
dic3['sentiment'].value_counts()

부정    9829
긍정    4871
Name: sentiment, dtype: int64

In [58]:
dic3.isnull().sum()

word         0
sentiment    1
dtype: int64

In [59]:
dic_s = pd.concat([dic1,dic2,dic3])

In [60]:
dic_s.dropna(axis=0, inplace=True)
dic_s.reset_index(drop=True, inplace=True)

In [61]:
dic_s.isnull().sum()

word         0
sentiment    0
dtype: int64

In [63]:
dic_s_n = dic_s[dic_s['sentiment'] == '부정']
dic_s_n.rename(columns={'word': '부정'}, inplace=True)
dic_s_p = dic_s[dic_s['sentiment'] == '긍정']
dic_s_p.rename(columns={'word': '긍정'}, inplace=True)

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [64]:
dic_s_n

,부정,sentiment
0,가난,부정
1,가난뱅이,부정
2,가난살이,부정
3,가난살이하다,부정
4,가난설음,부정
...,...,...
29390,오류,부정
29391,의혹,부정
29392,내팽개치다,부정
29393,횡령,부정


In [66]:
d1 = dic_s_n[['부정']].to_dict('list')
d2 = dic_s_p[['긍정']].to_dict('list')

In [ ]:
dic_final = dict(d1, **d2)
dic_final

In [68]:
import pickle

# save data
with open('dict.pickle','wb') as fw:
    pickle.dump(dic_final, fw)

In [69]:
import pickle

# load data
with open('dict.pickle', 'rb') as fr:
    dic_final = pickle.load(fr)

In [ ]:
dic_final

In [ ]:
df.to_dict('list')